In [8]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv('./annotated_nl_charts_pairs.csv')

# 统计good_charts_count的分布情况
distribution = df['good_charts_count'].value_counts().sort_index()

# 打印分布结果
print("good_charts_count的分布情况:")
print(distribution)

# 统计good_charts_count大于1的总数
count_greater_than_one = len(df[df['good_charts_count'] > 1])
print(f"\ngood_charts_count大于1的总数: {count_greater_than_one}")


good_charts_count的分布情况:
good_charts_count
0    38
1    69
2    45
3     4
4    57
Name: count, dtype: int64

good_charts_count大于1的总数: 106


In [10]:
import json

# 读取test_data.json文件
with open('./test_data.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

# 统计每个文件中good_generated_chart_list的长度
chart_list_lengths = [len(data['good_generated_chart_list']) for data in test_data.values()]

# 计算长度分布
length_distribution = pd.Series(chart_list_lengths).value_counts().sort_index()

print("good_generated_chart_list长度的分布情况:")
print(length_distribution)


good_generated_chart_list长度的分布情况:
2    45
3     4
4    57
Name: count, dtype: int64


In [11]:
# 找出good_generated_chart_list长度为5的文件
files_with_five_charts = [filename for filename, data in test_data.items() 
                         if len(data['good_generated_chart_list']) == 5]

print("good_generated_chart_list长度为5的文件:")
for filename in files_with_five_charts:
    print(filename)


good_generated_chart_list长度为5的文件:


In [13]:
from deepdiff import DeepDiff

def compare_vegalite(vl1, vl2):
    # 只对以下字段进行比较
    keys_to_compare = ["mark", "encoding", "transform"]
    sub_vl1 = {key: vl1.get(key) for key in keys_to_compare}
    sub_vl2 = {key: vl2.get(key) for key in keys_to_compare}
    
    # 比较字段的细节
    diff = DeepDiff(sub_vl1, sub_vl2, ignore_order=True)
    
    # 判断各字段的一致性
    consistency = {
        "mark": vl1.get("mark") == vl2.get("mark"),
        "field": vl1.get("encoding") == vl2.get("encoding"),
        "filter": "transform" not in diff or not any("filter" in d for d in diff.get("values_changed", {})),
        "aggregate": "values_changed" not in diff or "aggregate" not in diff.get("values_changed", {}),
        "sort": "values_changed" not in diff or "sort" not in diff.get("values_changed", {}),
        "bin": "values_changed" not in diff or "bin" not in diff.get("values_changed", {}),
    }
    
    # 判断 overall 一致性
    consistency["overall"] = all(consistency.values())
    return consistency

# 示例 JSON
vl1 = {
    "mark": "line",
    "encoding": {
        "x": {"field": "year", "type": "temporal"},
        "y": {"field": "gdpPercap", "type": "quantitative", "aggregate": "sum"},
        "color": {"field": "country", "type": "nominal"}
    },
    "transform": [
        {"filter": {"field": "country", "oneOf": ["Iceland", "Norway"]}},
        {"filter": {"field": "year", "range": [{"year": 1952}, {"year": 2017}]}}
    ]
}

vl2 = {
    "mark": "area",
    "encoding": {
        "x": {"field": "year", "type": "temporal"},
        "y": {"field": "gdpPercap", "type": "quantitative", "aggregate": "sum"},
        "color": {"field": "country", "type": "nominal"}
    },
    "transform": [
        {"filter": {"field": "country", "oneOf": ["Norway"]}},
        {"filter": {"field": "year", "range": [{"year": 1952}, {"year": 2017}]}}
    ]
}

# 对比结果
result = compare_vegalite(vl1, vl2)
print(result)
# 输出:
# {
#   'mark': False,
#   'field': True,
#   'filter': True,
#   'aggregate': True,
#   'sort': True,
#   'bin': True,
#   'overall': False
# }

{'mark': False, 'field': True, 'filter': True, 'aggregate': True, 'sort': True, 'bin': True, 'overall': False}


In [7]:
# 加载并输出文件长度
import json

with open('result/merged_result_gpt35.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    
print(f"merged_result_gpt35.json 文件中包含 {len(data)} 条数据")


merged_result_gpt35.json 文件中包含 105 条数据
